In [1]:
!pip install scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 40.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sklearn
import xgboost
print("scikit-learn version:", sklearn.__version__)
print("xgboost version:", xgboost.__version__)

scikit-learn version: 1.2.2
xgboost version: 2.1.3


In [ ]:
!pip freeze > requirements.txt

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# Load data
file_path = '/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/PINN/ain model/train_data_ain.csv'
data = pd.read_csv(file_path)

# Select 20,000 rows randomly
data_sampled = data.sample(n= 3000, random_state=42, replace=True)

# Assuming the last column is the target and others are features
X = data_sampled.iloc[:, :-1]
y = data_sampled.iloc[:, -1]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features for ML models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize result dictionary
best_params = {}

# XGBoost Model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
# xgb_model = XGBRegressor()
xgb_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.001, 0.01, 0.1],
     'subsample': [0.8, 1.0]
}
# xgb_grid = GridSearchCV(xgb_model, param_grid=xgb_param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
xgb_grid = GridSearchCV(estimator=xgb_model,
                        param_grid=xgb_param_grid,
                        scoring='neg_mean_squared_error',
                        cv=3,
                        verbose=1)
xgb_grid.fit(X_train_scaled, y_train)
best_params['XGBoost'] = xgb_grid.best_params_

# Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_grid = GridSearchCV(rf_model, rf_param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
rf_grid.fit(X_train_scaled, y_train)
best_params['Random Forest'] = rf_grid.best_params_

# Support Vector Machine (SVM)
svm_model = SVR()
svm_param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}
svm_grid = GridSearchCV(svm_model, svm_param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
svm_grid.fit(X_train_scaled, y_train)
best_params['SVM'] = svm_grid.best_params_

# ANN (MLPRegressor)
ann_model = MLPRegressor(max_iter=1000, random_state=42)
ann_param_grid = {
    'hidden_layer_sizes': [(40,40,40,40), (40,40,40,40,40,40), (40,40,40,40,40,40,40,40)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}
ann_grid = GridSearchCV(ann_model, ann_param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
ann_grid.fit(X_train_scaled, y_train)
best_params['ANN'] = ann_grid.best_params_

# Evaluate models and display best parameters
print("Best Parameters for Each Model:")
for model, params in best_params.items():
    print(f"{model}: {params}")

# Optional: Evaluate on test set
xgb_best = xgb_grid.best_estimator_
rf_best = rf_grid.best_estimator_
svm_best = svm_grid.best_estimator_
ann_best = ann_grid.best_estimator_

xgb_pred = xgb_best.predict(X_test_scaled)
rf_pred = rf_best.predict(X_test_scaled)
svm_pred = svm_best.predict(X_test_scaled)
ann_pred = ann_best.predict(X_test_scaled)

print("Test RMSE Scores:")
print(f"XGBoost: {np.sqrt(mean_squared_error(y_test, xgb_pred))}")
print(f"Random Forest: {np.sqrt(mean_squared_error(y_test, rf_pred))}")
print(f"SVM: {np.sqrt(mean_squared_error(y_test, svm_pred))}")
print(f"ANN: {np.sqrt(mean_squared_error(y_test, ann_pred))}")


Fitting 3 folds for each of 54 candidates, totalling 162 fits
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Fitting 3 folds for each of 36 candidates, totalling 108 fits


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

Best Parameters for Each Model:
XGBoost: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 150, 'subsample': 1.0}
Random Forest: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
SVM: {'C': 10, 'epsilon': 1, 'kernel': 'rbf'}
ANN: {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (40, 40, 40, 40, 40, 40, 40, 40), 'learning_rate': 'constant'}
Test RMSE Scores:
XGBoost: 1.233373455358622
Random Forest: 5.00923695219891
SVM: 3.5197089188103177
ANN: 6.1741082620083


In [ ]:
# # prompt: give me summary of the datasets in stored in csv  in a tabular formtrain_data_ain.csv

# import pandas as pd

# # Load the dataframe (assuming 'data' is already loaded as in your provided code)
# # If 'data' is not available, replace this with pd.read_csv(...) to load it.
# file_path = '/content/drive/MyDrive/Colab Notebooks/PINN/ain model/train_data_ain.csv'
# data = pd.read_csv(file_path)


# # Display the summary statistics of the dataframe
# summary = data.describe(include='all').T  # Transpose the table for better readability

# # Display the summary
# summary

,count,mean,std,min,25%,50%,75%,max
x,55000.0,13435.464466,5875.730451,623.932991,8859.848473,13601.739204,18343.629936,25331.679436
y,55000.0,20173.559318,8360.924404,3847.414749,13776.227005,19733.514359,26683.682938,37357.156113
t,55000.0,1176.824893,899.765019,0.000000,245.000137,1095.999512,1947.000488,2556.001953
K,55000.0,483.398914,658.308417,5.000000,65.000000,86.400002,864.000000,3456.000000
W,55000.0,-0.000006,0.000284,-0.008322,0.000000,0.000009,0.000033,0.003596
S,55000.0,0.055267,0.024684,0.010000,0.050000,0.050000,0.050000,0.170000
h,55000.0,220.825322,32.600176,-888.000000,203.975220,225.957085,237.071899,285.519501


from matplotlib import pyplot as plt
summary['mean'].plot(kind='hist', bins=20, title='mean')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
summary['std'].plot(kind='hist', bins=20, title='std')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
summary['min'].plot(kind='hist', bins=20, title='min')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
summary['25%'].plot(kind='hist', bins=20, title='25%')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['count']
  ys = series['25%']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = summary.sort_values('count', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('count')
_ = plt.ylabel('25%')

from matplotlib import pyplot as plt
summary['mean'].plot(kind='line', figsize=(8, 4), title='mean')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
summary['std'].plot(kind='line', figsize=(8, 4), title='std')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
summary['min'].plot(kind='line', figsize=(8, 4), title='min')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
summary['25%'].plot(kind='line', figsize=(8, 4), title='25%')
plt.gca().spines[['top', 'right']].set_visible(False)

In [6]:
# prompt: save the models to .pkl file at google drive

import pickle

# Save the models to .pkl files in Google Drive
models = {
    'xgb_model6': xgb_best,
    'random_forest_model6': rf_best,
    'svm_model6': svm_best,
    'ann_model6': ann_best,
    'scaler6': scaler
}

for model_name, model in models.items():
    filename = f'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/PINN/ain model/{model_name}.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

In [4]:
# prompt: save the models to .pkl file at google drive. also save the test rmse

# Save the test RMSE scores
import os
results_dir = '/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/PINN/ain model/results'
!mkdir -p "{results_dir}"

with open(os.path.join(results_dir, 'test_rmse6.txt'), 'w') as f:
    f.write(f"XGBoost: {np.sqrt(mean_squared_error(y_test, xgb_pred))}\n")
    f.write(f"SVM: {np.sqrt(mean_squared_error(y_test, svm_pred))}\n")
    f.write(f"Random Forest: {np.sqrt(mean_squared_error(y_test, rf_pred))}\n")
    f.write(f"ANN: {np.sqrt(mean_squared_error(y_test, ann_pred))}\n")

In [7]:
# prompt: read file 'combined_test_data_ain.csv' use the first six columns as input and predict the 'h' corresponding to these variables, using all the three tuned models. Store the input and output (column name: 'h_xgb', 'h_bayesian' and 'h_ann') into file 'final_ml_model.csv'

import pandas as pd
import numpy as np
import pickle

# Load the saved models and scaler
models = {}
for model_name in ['xgb_model6', 'svm_model6' , 'random_forest_model6', 'ann_model6', 'scaler6']:
    filename = f'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/PINN/ain model/{model_name}.pkl'
    with open(filename, 'rb') as file:
        models[model_name] = pickle.load(file)

# Load the test data
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/PINN/ain model/combined_test_data_ain.csv')
X_test = test_data.iloc[:, :6]  # Select the first six columns as input

# Scale the input features using the saved scaler
X_test_scaled = models['scaler6'].transform(X_test)

# Make predictions using the loaded models
xgb_pred = models['xgb_model6'].predict(X_test_scaled)
svm_pred = models['svm_model6'].predict(X_test_scaled)
rf_pred = models['random_forest_model6'].predict(X_test_scaled)
ann_pred = models['ann_model6'].predict(X_test_scaled)


# Create a DataFrame with the input features and predictions
results_df = pd.DataFrame(X_test)
results_df['h_xgb'] = xgb_pred
results_df['h_svm'] = svm_pred
results_df['h_rf'] = rf_pred
results_df['h_ann'] = ann_pred

# Save the results to a CSV file
results_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/PINN/ain model/final_ml_model6.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive
